In [1]:
import os,shutil
from PIL import Image
import numpy as np
import cv2
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA

In [2]:
cur_dir = os.getcwd()

dataset_dir = os.path.join(cur_dir,"Dataset")

labels_dir = os.path.join(dataset_dir,"labels")
frames_dir = os.path.join(dataset_dir,"frames")

n_lecs = len(os.listdir(labels_dir))

listOfLectures = []

for lec in os.listdir(frames_dir):
    frame_list = []
    lec_path = os.path.join(frames_dir,lec)
    for frame in os.listdir(lec_path):
        frame_list.append([os.path.join(lec_path,frame),])
    listOfLectures.append(frame_list)

# print(listOfLectures[:2])

In [3]:
lec_num = 0
for csv in os.listdir(labels_dir):
    csv_path = os.path.join(labels_dir,csv)
    with open(csv_path,'r') as input:
        data = input.read()
        data = data.split()
        for i in range(len(data)):
            listOfLectures[lec_num][i].append(int(data[i]))
    lec_num+=1

In [ ]:
# listOfLectures : list of lectures where each lecture contains a list of form [frame_number, whether hit or miss ]
im = cv2.imread(listOfLectures[0][134][0],0)
# cv2.imshow('image',im)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# print(im.dtype)
x = cv2.calcHist([im],[0],None,[256],[0,256])
# print(x)
# plt.hist(im.ravel(),256,[0,256])
# plt.show()

In [4]:
def ECR(frame, next_frame, width, height, crop=True, dilate_rate = 5):
    safe_div = lambda x,y: 0 if y == 0 else x / y
    if crop:
        startY = int(height * 0.1)
        endY = int(height * 0.9)
        startX = int(width * 0.1)
        endX = int(width * 0.9)
        frame = frame[startY:endY, startX:endX]
        next_frame = next_frame[startY:endY, startX:endX]

    edge = cv2.Canny(frame, 0, 200)
    dilated = cv2.dilate(edge, np.ones((dilate_rate, dilate_rate)))
    inverted = (255 - dilated)
    edge2 = cv2.Canny(next_frame, 0, 200)
    dilated2 = cv2.dilate(edge2, np.ones((dilate_rate, dilate_rate)))
    inverted2 = (255 - dilated2)
    log_and1 = (edge2 & inverted)
    log_and2 = (edge & inverted2)
    pixels_sum_new = np.sum(edge)
    pixels_sum_old = np.sum(edge2)
    out_pixels = np.sum(log_and1)
    in_pixels = np.sum(log_and2)
    return max(safe_div(float(in_pixels),float(pixels_sum_new)), safe_div(float(out_pixels),float(pixels_sum_old)))

In [5]:
for i in range(len(listOfLectures)):
    # i is lec number
    
    pca = PCA(n_components=5)
    sample_image = cv2.imread(listOfLectures[i][0][0],0)
    X= sample_image.ravel().reshape((1,-1))
#     print(X.shape)
    for j in range(1,len(listOfLectures[i])-1):
        im = cv2.imread(listOfLectures[i][j][0],0).ravel().reshape((1,-1))
#       print(im.shape)
        X = np.vstack((X,im))
    pca.fit(X)
    X = pca.transform(X)
        
#     print(X.shape)

    
    for j in range(len(listOfLectures[i])-1):
        # j is frame nu"mber
        frame_params = []
        f_info = listOfLectures[i][j]
        print(f_info)
        f_info_next = listOfLectures[i][j+1]
        # As of now , list of form [Path,missOrHit]
        im = cv2.imread(f_info[0],0) # Grayscale image of frame
        im_n = cv2.imread(f_info_next[0],0) # Grayscale image of frame
        
        sad = np.sum(np.abs(im_n-im))
        frame_params.append(sad)
        
        ecr = ECR(im,im_n,im.shape[1],im.shape[0])
        frame_params.append(ecr)
        
        im_h = cv2.calcHist([im],[0],None,[256],[0,256])
        im_nh = cv2.calcHist([im_n],[0],None,[256],[0,256])
        hd = np.sum(np.abs(im_h-im_nh))
        frame_params.append(hd)
#         print(frame_params)
        print(f_info)
        f_info.append(frame_params)
        print(f_info)
        f_info.append(X[j])
        print(f_info)
        break        
    break
        
        

['C:\\Users\\Siddharth Khera\\Dropbox\\This Sem\\ELL888 - Deep Learning\\assn2\\Dataset\\frames\\lec1\\l1s00000.jpg', 0]
['C:\\Users\\Siddharth Khera\\Dropbox\\This Sem\\ELL888 - Deep Learning\\assn2\\Dataset\\frames\\lec1\\l1s00000.jpg', 0]
['C:\\Users\\Siddharth Khera\\Dropbox\\This Sem\\ELL888 - Deep Learning\\assn2\\Dataset\\frames\\lec1\\l1s00000.jpg', 0, [13095882, 0.9908933217692975, 38638.0]]
['C:\\Users\\Siddharth Khera\\Dropbox\\This Sem\\ELL888 - Deep Learning\\assn2\\Dataset\\frames\\lec1\\l1s00000.jpg', 0, [13095882, 0.9908933217692975, 38638.0], array([-10702.11322391,   -780.05517163,     51.03489705,  -2792.60960977,
          738.34880268])]
